In [45]:
import numpy as np
from scipy.spatial import ConvexHull

def generar_poliedro():
    # Rango de muestreo
    xs = np.linspace(0, 12, 50)
    ys = np.linspace(0, 12, 50)
    zs = np.linspace(0, 12, 50)
    X, Y, Z = np.meshgrid(xs, ys, zs)
    points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

    # Restricciones lineales
    factibles = []
    for (x,y,z) in points:
        if (x >= 0 and y >= 0 and z >= 0 and
            x + y + z <= 12 and
            2*x + y <= 14 and
            x + 3*z <= 18 and
            y + 2*z <= 14):
            factibles.append([x,y,z])
    factibles = np.array(factibles)

    # Poliedro factible (envolvente convexa)
    hull = ConvexHull(factibles)

    return factibles, hull


In [46]:
# Función objetivo
def f(x,y,z):
    return 2*x + y + 3*z

In [47]:
import plotly.graph_objects as go

def graficar_poliedro(factibles, hull):
    i, j, k = hull.simplices.T
    vertices = factibles[hull.vertices]

    fig = go.Figure()

    # Poliedro
    fig.add_trace(go.Mesh3d(
        x=factibles[:,0],
        y=factibles[:,1],
        z=factibles[:,2],
        i=i, j=j, k=k,
        opacity=0.5,
        color="cyan",
        name="Poliedro factible"
    ))

    # Vértices
    fig.add_trace(go.Scatter3d(
        x=vertices[:,0],
        y=vertices[:,1],
        z=vertices[:,2],
        mode="markers",
        marker=dict(size=5, color="red"),
        name="Vértices"
    ))

    fig.update_layout(
        scene=dict(
            xaxis=dict(title="x", range=[-20, 20]),
            yaxis=dict(title="y", range=[-20, 20]),
            zaxis=dict(title="z", range=[-20, 20])
        ),
        margin=dict(l=40, r=40, b=40, t=40),
        title="Poliedro factible en 3D (Plotly)"
    )

    fig.show()


In [48]:
factibles, hull = generar_poliedro()
graficar_poliedro(factibles, hull)


In [49]:
import numpy as np
from scipy.spatial import ConvexHull
import plotly.graph_objects as go

def generar_poliedro():
    # Rango de muestreo
    xs = np.linspace(0, 12, 50)
    ys = np.linspace(0, 12, 50)
    zs = np.linspace(0, 12, 50)
    X, Y, Z = np.meshgrid(xs, ys, zs)
    points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

    # Restricciones lineales
    factibles = []
    for (x,y,z) in points:
        if (x >= 0 and y >= 0 and z >= 0 and
            x + y + z <= 12 and
            2*x + y <= 14 and
            x + 3*z <= 18 and
            y + 2*z <= 14):
            factibles.append([x,y,z])
    factibles = np.array(factibles)

    # Poliedro factible (envolvente convexa)
    hull = ConvexHull(factibles)

    return factibles, hull

# Función objetivo
def f(x,y,z):
    return 2*x + y + 3*z

def graficar_poliedro_coloreado(factibles, hull):
    i, j, k = hull.simplices.T
    vertices = factibles[hull.vertices]

    # Evaluar la función objetivo en cada vértice
    valores = np.array([f(x,y,z) for x,y,z in vertices])

    fig = go.Figure()

    # Poliedro (transparente)
    fig.add_trace(go.Mesh3d(
        x=factibles[:,0],
        y=factibles[:,1],
        z=factibles[:,2],
        i=i, j=j, k=k,
        opacity=0.2,
        color="lightblue",
        name="Poliedro factible"
    ))

    # Vértices coloreados por el valor objetivo
    fig.add_trace(go.Scatter3d(
        x=vertices[:,0],
        y=vertices[:,1],
        z=vertices[:,2],
        mode="markers",
        marker=dict(
            size=6,
            color=valores,      # <-- valores de f(x,y,z)
            colorscale="Viridis", # puedes probar "Jet", "Plasma", etc.
            colorbar=dict(title="f(x,y,z)")
        ),
        text=[f"f={val:.2f}" for val in valores],
        name="Vértices"
    ))

    fig.update_layout(
        scene=dict(
            xaxis=dict(title="x", range=[-2, 14]),
            yaxis=dict(title="y", range=[-2, 14]),
            zaxis=dict(title="z", range=[-2, 14])
        ),
        margin=dict(l=40, r=40, b=40, t=40),
        title="Poliedro factible en 3D coloreado por f(x,y,z)"
    )

    fig.show()


In [50]:
factibles, hull = generar_poliedro()
graficar_poliedro_coloreado(factibles, hull)


In [51]:
import pandas as pd

def listar_vertices():
    factibles, hull = generar_poliedro()
    vertices = factibles[hull.vertices]

    df_vertices = pd.DataFrame(vertices, columns=["x", "y", "z"])
    df_vertices = df_vertices.round(2).drop_duplicates().reset_index(drop=True)

    return df_vertices


In [52]:
tabla_vertices = listar_vertices()
print(tabla_vertices)


       x      y     z
0   0.00   0.00  0.00
1   0.00   0.00  5.88
2   0.24   0.00  5.88
3   6.86   0.00  0.00
4   6.86   0.00  3.67
5   6.86   0.24  0.00
6   6.86   0.24  3.67
7   6.12   1.71  3.92
8   0.00   2.20  5.88
9   0.24   2.20  5.88
10  5.88   2.20  3.92
11  5.39   2.45  4.16
12  2.45   3.67  5.14
13  3.18   3.92  4.90
14  2.94   4.16  4.90
15  0.00  10.04  1.96
16  1.96  10.04  0.00
17  0.00  12.00  0.00


In [53]:
import numpy as np

def simplex_una_iteracion():
    # Obtener vértices del poliedro
    factibles, hull = generar_poliedro()
    vertices = factibles[hull.vertices]

    # Evaluar objetivo en cada vértice
    valores = [f(x,y,z) for x,y,z in vertices]

    # Elegir vértice inicial (el de menor x,y,z por ejemplo)
    idx_ini = np.argmin(vertices[:,0] + vertices[:,1] + vertices[:,2])
    v_ini = vertices[idx_ini]
    val_ini = f(*v_ini)

    # Simular vecinos (todos los demás vértices por simplicidad)
    mejoras = []
    for i, v in enumerate(vertices):
        if i != idx_ini:
            val = f(*v)
            if val > val_ini:
                mejoras.append((val, v))

    if mejoras:
        # Escoger el que más mejora
        val_mejor, v_mejor = max(mejoras, key=lambda x: x[0])
        return {
            "inicio": (v_ini, val_ini),
            "siguiente": (v_mejor, val_mejor)
        }
    else:
        return {
            "inicio": (v_ini, val_ini),
            "siguiente": None
        }


In [54]:
import numpy as np
import plotly.graph_objects as go

def graficar_poliedro_con_camino(factibles, hull, camino_df, f_obj):
    i, j, k = hull.simplices.T
    vertices = factibles[hull.vertices]

    # Evaluamos la función objetivo en cada vértice
    valores = np.array([f_obj(x,y,z) for x,y,z in vertices])

    fig = go.Figure()

    # Poliedro factible (transparente)
    fig.add_trace(go.Mesh3d(
        x=factibles[:,0],
        y=factibles[:,1],
        z=factibles[:,2],
        i=i, j=j, k=k,
        opacity=0.2,
        color="lightblue",
        name="Poliedro factible"
    ))

    # Vértices coloreados por f(x,y,z)
    fig.add_trace(go.Scatter3d(
        x=vertices[:,0],
        y=vertices[:,1],
        z=vertices[:,2],
        mode="markers",
        marker=dict(
            size=6,
            color=valores,
            colorscale="Viridis",
            colorbar=dict(title="f(x,y,z)")
        ),
        text=[f"f={val:.2f}" for val in valores],
        name="Vértices"
    ))

    # Camino del simplex (línea azul + puntos azules)
    fig.add_trace(go.Scatter3d(
        x=camino_df["x"],
        y=camino_df["y"],
        z=camino_df["z"],
        mode="lines+markers",
        line=dict(color="blue", width=4),
        marker=dict(size=7, color="blue"),
        text=[str(p) for p in camino_df["paso"]],  # numerar pasos
        textposition="top center",
        name="Camino simplex"
    ))

    # Último punto = óptimo (verde grande)
    v_opt = camino_df.iloc[-1]
    fig.add_trace(go.Scatter3d(
        x=[v_opt["x"]],
        y=[v_opt["y"]],
        z=[v_opt["z"]],
        mode="markers+text",
        marker=dict(size=12, color="green", symbol="diamond"),
        text=[f"Óptimo<br>f={v_opt['valor_objetivo']:.2f}"],
        textposition="top center",
        name="Óptimo"
    ))

    fig.update_layout(
        scene=dict(
            xaxis=dict(title="x", range=[-2, 14]),
            yaxis=dict(title="y", range=[-2, 14]),
            zaxis=dict(title="z", range=[-2, 14])
        ),
        margin=dict(l=40, r=40, b=40, t=40),
        title="Poliedro factible + Camino del simplex"
    )

    fig.show()


In [55]:
from collections import defaultdict

def construir_adyacencia_local(hull):
    """
    Construye adyacencia entre índices locales (0..len(hull.vertices)-1)
    en lugar de índices globales.
    """
    # Mapeo de índice global -> índice local
    global2local = {g: i for i, g in enumerate(hull.vertices)}
    adj = defaultdict(set)

    for simplex in hull.simplices:
        for i in range(len(simplex)):
            for j in range(i+1, len(simplex)):
                gi, gj = simplex[i], simplex[j]
                if gi in global2local and gj in global2local:
                    li, lj = global2local[gi], global2local[gj]
                    adj[li].add(lj)
                    adj[lj].add(li)
    return adj

def simplex_camino_por_adyacencia():
    factibles, hull = generar_poliedro()
    vertices = factibles[hull.vertices]  # solo vértices extremos
    valores = np.array([f(x,y,z) for x,y,z in vertices])

    # Construir grafo de adyacencia en términos de índices locales
    adj = construir_adyacencia_local(hull)

    # Vértice inicial (menor suma coords)
    idx_actual = np.argmin(vertices.sum(axis=1))
    camino = []

    while True:
        v_actual = vertices[idx_actual]
        val_actual = valores[idx_actual]
        camino.append((v_actual, val_actual))

        # Vecinos adyacentes
        vecinos = adj[idx_actual]
        mejoras = []
        for i in vecinos:
            if valores[i] > val_actual:
                mejoras.append((valores[i], i))

        if not mejoras:
            break  # óptimo alcanzado

        val_mejor, idx_mejor = max(mejoras, key=lambda x: x[0])
        if val_mejor <= val_actual:
            break

        idx_actual = idx_mejor

    # Guardar trayectoria como DataFrame
    datos = []
    for paso, (v, val) in enumerate(camino):
        datos.append({"paso": paso, "x": v[0], "y": v[1], "z": v[2], "valor_objetivo": val})

    return pd.DataFrame(datos)


In [56]:
factibles, hull = generar_poliedro()
camino_df = simplex_camino_por_adyacencia()   # tu versión simplex
graficar_poliedro_con_camino(factibles, hull, camino_df, f)
